# Compute PSD, SNR, and Z-Score of Pilot Data

## Goals:
1. **Data Import**
    - Import the preprocessed data from npz and json files

2. **Feature Extraction**
    - Calculate the Power Spectral Density (PSD) for each epoch.
    - Calculate the Signal-to-Noise Ratio (SNR) for each epoch (for use in stats)
    - Calculate the Z-score for each epoch (used to determine signal cutoff):
        * **Formula**:   PSD Z-score = (PSD(single trial) - Mean PSD of baseline trials) / Std PSD of baseline trials
        
3. **Data Formatting**
    - Save the data and export to an excel sheet
        * Get the average of all epochs for each unique stimulus

# Import Libraries

In [1]:
# Standard libraries
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Custom libraries
from Functions import processing

%matplotlib qt

# Import Epoched Data and Settings

In [2]:
# Load list of files to import
files = [  
    "sub-P001_ses-S001_task-T1_run-001_eeg"   
]

# Get unique subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables to store EEG data and settings
eeg_epochs = [None] * len(files)
settings = [None] * len(files)
npy_data = [None] * len(files)

# Import data
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz) we need to use the np.load function 
    loaded_data = np.load(f"Data\\Masters_testing\\{file}.npz", allow_pickle=True)

    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}

    # Import settings
    with open(f"Data\\Masters_testing\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)

    # Import npy data
    npy_data[f] = np.load(f"Data\\Masters_testing\\{file}.npy", allow_pickle=True)

# Compute PSD for all Epochs

Make sure to include baseline epochs for later calculation of Z-score
Compute PSD for every epoch (and average for repeated labels after)

In [3]:
# PSD settings
window_size = 10  # Length of window for PSD [sec], gives a frequency resolution of 0.5 Hz so I will get a psd value for every 0.5 Hz

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)  # Preallocate to list in case not all files have the same number of channels

# Compute PSD for each file
for f, file in enumerate(files):
    # Preallocate lists to store PSD results for each stimulus
    eeg_f[f] = {}
    eeg_pxx[f] = {}

    # Compute PSD for each stimulus
    for stim_label, epochs in eeg_epochs[f].items():
        # Preallocate lists to store PSD results for each epoch
        eeg_f[f][stim_label] = []
        eeg_pxx[f][stim_label] = []

        # Compute PSD for each epoch
        for epoch in epochs:
            f_values, pxx_values = signal.welch(
                x=epoch,
                fs=settings[f]["eeg_srate"],
                nperseg=window_size * settings[f]["eeg_srate"]
            )
            eeg_f[f][stim_label].append(f_values)
            eeg_pxx[f][stim_label].append(pxx_values)

        # Convert lists to arrays for consistency
        eeg_f[f][stim_label] = np.array(eeg_f[f][stim_label])
        eeg_pxx[f][stim_label] = np.array(eeg_pxx[f][stim_label])

# Print the shape of the PSD results for each stimulus
for f, file in enumerate(files):
    print(f"File: {file}")
    for stim_label in eeg_f[f].keys():
        print(f"  Stimulus: {stim_label}, PSD shape: {eeg_pxx[f][stim_label].shape}")

File: sub-P001_ses-S001_task-T1_run-001_eeg
  Stimulus: Contrast1Size1, PSD shape: (2, 13, 1281)
  Stimulus: Contrast1Size2, PSD shape: (2, 13, 1281)
  Stimulus: Contrast1Size3, PSD shape: (2, 13, 1281)
  Stimulus: Contrast2Size1, PSD shape: (3, 13, 1281)
  Stimulus: Contrast2Size2, PSD shape: (4, 13, 1281)
  Stimulus: Contrast2Size3, PSD shape: (3, 13, 1281)
  Stimulus: Contrast3Size1, PSD shape: (3, 13, 1281)
  Stimulus: Contrast3Size2, PSD shape: (3, 13, 1281)
  Stimulus: Contrast3Size3, PSD shape: (4, 13, 1281)
  Stimulus: Contrast4Size1, PSD shape: (5, 13, 1281)
  Stimulus: Contrast4Size2, PSD shape: (3, 13, 1281)
  Stimulus: Contrast4Size3, PSD shape: (6, 13, 1281)


# Compute PSD for Resting State (Eyes Open)

In [4]:
npy_f = [None] * len(files)
npy_pxx = [None] * len(files)

for f, file in enumerate(files):
# Compute PSD for npy_data
    npy_f[f] = []
    npy_pxx[f] = []
    for epoch in npy_data[f]:
        f_values, pxx_values = signal.welch(
            x=epoch,
            fs=settings[f]["eeg_srate"],
            nperseg=window_size * settings[f]["eeg_srate"]
        )
        npy_f[f].append(f_values)
        npy_pxx[f].append(pxx_values)

    # Convert lists to arrays for consistency
    npy_f[f] = np.array(npy_f[f])
    npy_pxx[f] = np.array(npy_pxx[f])

# Print the shape of the PSD results for each stimulus
for f, file in enumerate(files):
    print(f"File: {file}")
    print(f"NPY data PSD shape: {npy_pxx[f].shape}")

File: sub-P001_ses-S001_task-T1_run-001_eeg
NPY data PSD shape: (1, 1, 13, 1281)


# Visualize PSD

In [5]:
import matplotlib.pyplot as plt

# Plot settings
plot_psd = True   # Enable to see plots
f_limits = [4, 35]  # Frequency limits for the plots [min, max][Hz]
file_to_plot = 0    # Select index of file to be plotted 
stimulus_index = 2  # Select index of stimulus to be plotted

if plot_psd:
    # Get the stimulus label by index
    stim_label = list(eeg_f[file_to_plot].keys())[stimulus_index]
    
    # Mask for frequency limits
    fmask = (eeg_f[file_to_plot][stim_label][0] >= f_limits[0]) & (eeg_f[file_to_plot][stim_label][0] <= f_limits[1])
    temp_freq = eeg_f[file_to_plot][stim_label][0][fmask]

    # Number of epochs
    num_epochs = len(eeg_pxx[file_to_plot][stim_label])

    # Create subplots dynamically based on the number of epochs plus one for the average
    fig, ax = plt.subplots(num_epochs + 1, 1, figsize=(10, 3 * (num_epochs + 1)))
    fig.suptitle(f'PSD for Stimulus: {stim_label} in File: {files[file_to_plot]}')

    # Plot PSD for each epoch
    for epoch_idx in range(num_epochs):
        temp_mean = np.mean(eeg_pxx[file_to_plot][stim_label][epoch_idx], axis=0)[fmask]
        temp_sd = np.std(eeg_pxx[file_to_plot][stim_label][epoch_idx], axis=0)[fmask]

        ax[epoch_idx].plot(temp_freq, temp_mean, '-')
        #ax[epoch_idx].fill_between(temp_freq, temp_mean - temp_sd, temp_mean + temp_sd, alpha=0.3)
        ax[epoch_idx].set_title(f'Epoch {epoch_idx + 1}')
        ax[epoch_idx].set_xlim(f_limits)
        ax[epoch_idx].set_ylabel("PXX [$\mu$V$^2$/Hz]")
        if epoch_idx == num_epochs - 1:
            ax[epoch_idx].set_xlabel("Frequency [Hz]")
        # Add vertical lines
        for x in [5, 10, 20, 30]:
            ax[epoch_idx].axvline(x=x, color='r', linestyle='--')
        ax[epoch_idx].axvline(x=12, color='g', linestyle='--')
        ax[epoch_idx].axvline(x=11, color='pink', linestyle='--')
        ax[epoch_idx].axvline(x=11.5, color='orange', linestyle='--')



    # Plot average PSD across all epochs
    avg_pxx = np.mean(eeg_pxx[file_to_plot][stim_label], axis=0)
    avg_mean = np.mean(avg_pxx, axis=0)[fmask]
    avg_sd = np.std(avg_pxx, axis=0)[fmask]

    ax[num_epochs].plot(temp_freq, avg_mean, '-')
    #ax[num_epochs].fill_between(temp_freq, avg_mean - avg_sd, avg_mean + avg_sd, alpha=0.3)
    ax[num_epochs].set_title('Average PSD across all epochs')
    ax[num_epochs].set_xlim(f_limits)
    ax[num_epochs].set_ylabel("PXX [$\mu$V$^2$/Hz]")
    ax[num_epochs].set_xlabel("Frequency [Hz]")
    # Add vertical lines
    for x in [5, 10, 20, 30]:
        ax[num_epochs].axvline(x=x, color='r', linestyle='--')
    ax[num_epochs].axvline(x=12, color='g', linestyle='--')
    ax[num_epochs].axvline(x=11, color='pink', linestyle='--')
    ax[num_epochs].axvline(x=11.5, color='orange', linestyle='--')



    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Visualize PSD for Resting

In [6]:
# Visualization settings
plot_psd = True   # Enable to see plots
f_limits = [4, 35]  # Frequency limits for the plots [min, max][Hz]
file_to_plot = 0    # Select index of file to be plotted 

if plot_psd:
    # Plot NPY data PSD
    fig, ax = plt.subplots(1, 1, figsize=(10, 3))
    fig.suptitle(f'PSD for Resting State Eyes Open Data in File: {files[file_to_plot]}')

    # Mask for frequency limits
    fmask = (npy_f[file_to_plot][0] >= f_limits[0]) & (npy_f[file_to_plot][0] <= f_limits[1])
    temp_freq = npy_f[file_to_plot][0][fmask]

    # Compute mean and standard deviation for the single epoch
    temp_mean = np.mean(npy_pxx[file_to_plot][0][0], axis=0)[fmask]
    temp_sd = np.std(npy_pxx[file_to_plot][0][0], axis=0)[fmask]

    ax.plot(temp_freq, temp_mean, '-')
    ax.set_title('Resting State Eyes Open PSD')
    ax.set_xlim(f_limits)
    ax.set_ylabel("PXX [$\mu$V$^2$/Hz]")
    ax.set_xlabel("Frequency [Hz]")
    ax.axvline(x=16, color='g', linestyle='-')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Compute SNR for all Epochs


# Compute Z-score for all Epochs

# Save Data